<a href="https://colab.research.google.com/github/pablillo77/data_science_1/blob/main/Clase_07_datos_faltantes_desde%20colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Manipulacion de faltantes

In [1]:
import pandas as pd #importamos librerias
import numpy as np
url='https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv'
df= pd.read_csv(url,sep=',', header=None)
df.head()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


**Variables incorporadas**

    0. Numero de veces embarazada (NEMB).
    1. Concentracion de plasma de glucosa (GLU)
    2. Presion arterial diastolica en mm Hg (PART).
    3. Grosor de piel en triceps en mm (GROS).
    4. 2-Hour serum insulin en mu U/ml (HUR).
    5. BMI (peso kg/(altura en m)^2 en (BMI).
    6. Funcion de prediccion de Diabetes (FPRED)
    7. Edad (años)  (AGE).
    8. Variable de clase (0 or 1)  (CLASS).


In [2]:
# renombramos las columnas
df= df.rename(columns={0: "NEMB", 1: "GLU", 2: "PART",3:"GROS",4:"HUR",5:"BMI",6:"FPRED",7:"AGE",8:"CLASS"})
df.head()

,NEMB,GLU,PART,GROS,HUR,BMI,FPRED,AGE,CLASS
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   NEMB    768 non-null    int64  
 1   GLU     768 non-null    int64  
 2   PART    768 non-null    int64  
 3   GROS    768 non-null    int64  
 4   HUR     768 non-null    int64  
 5   BMI     768 non-null    float64
 6   FPRED   768 non-null    float64
 7   AGE     768 non-null    int64  
 8   CLASS   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [13]:
df.shape

(768, 9)

In [5]:
df.describe()

,NEMB,GLU,PART,GROS,HUR,BMI,FPRED,AGE,CLASS
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [6]:
resumen=df.describe().round(1)
resumen

,NEMB,GLU,PART,GROS,HUR,BMI,FPRED,AGE,CLASS
count,768.0,768.0,768.0,768.0,768.0,768.0,768.0,768.0,768.0
mean,3.8,120.9,69.1,20.5,79.8,32.0,0.5,33.2,0.3
std,3.4,32.0,19.4,16.0,115.2,7.9,0.3,11.8,0.5
min,0.0,0.0,0.0,0.0,0.0,0.0,0.1,21.0,0.0
25%,1.0,99.0,62.0,0.0,0.0,27.3,0.2,24.0,0.0
50%,3.0,117.0,72.0,23.0,30.5,32.0,0.4,29.0,0.0
75%,6.0,140.2,80.0,32.0,127.2,36.6,0.6,41.0,1.0
max,17.0,199.0,122.0,99.0,846.0,67.1,2.4,81.0,1.0


Se observan muchas columnas con valores minimos de 0 lo cual tiene poco sentido practico como por ejemplo:

    1. Concentracion de plasma de glucosa (GLU)
    2. Presion arterial diastolica en mm Hg (PART).
    3. Grosor de piel en triceps en mm (GROS).
    4. 2-Hour serum insulin en mu U/ml (HUR).
    5. BMI (peso kg/(altura en m)^2 en (BMI).


Calculemos cuantos valores problematicos tenemos

In [11]:
# contar el numero de valores que son 0 por columnas
nun_missing = (df[['GLU','PART','GROS','HUR','BMI']] == 0).sum()
print(nun_missing)


GLU       5
PART     35
GROS    227
HUR     374
BMI      11
dtype: int64


Reemplazar estos valores por NaN

In [24]:
# replace '0' values with 'nan'
df[['GLU','PART','GROS','HUR','BMI']] = df[['GLU','PART','GROS','HUR','BMI']].replace(0, np.nan)
# count the number of nan values in each column
print(df.isnull().sum())

NEMB       0
GLU        5
PART      35
GROS     227
HUR      374
BMI       11
FPRED      0
AGE        0
CLASS      0
dtype: int64


In [25]:
df.loc[df.GLU.isna()]

,NEMB,GLU,PART,GROS,HUR,BMI,FPRED,AGE,CLASS
75,1,NaN,48.0,20.0,NaN,24.7,0.140,22,0
182,1,NaN,74.0,20.0,23.0,27.7,0.299,21,0
342,1,NaN,68.0,35.0,NaN,32.0,0.389,22,0
349,5,NaN,80.0,32.0,NaN,41.0,0.346,37,1
502,6,NaN,68.0,41.0,NaN,39.0,0.727,41,1


In [26]:
"""
La única diferencia entre isna() e isnull() es su nombre. isnull() es el nombre original de la función que se ha
utilizado en versiones anteriores de Pandas, mientras que isna() es un alias que se ha agregado luego
para mejorar la legibilidad del código.
"""

print(df.isna().sum())

NEMB       0
GLU        5
PART      35
GROS     227
HUR      374
BMI       11
FPRED      0
AGE        0
CLASS      0
dtype: int64


In [27]:
#que pasa si necesitamos reemplazar valores que son fisicamente sospechosos
url='https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv'
df2= pd.read_csv(url,sep=',', header=None)
df2= df2.rename(columns={0: "NEMB", 1: "GLU", 2: "PART",3:"GROS",4:"HUR",5:"BMI",6:"FPRED",7:"AGE",8:"CLASS"})
nun_missing_2 = (df2['GLU'] < 45).sum()
print(nun_missing_2)

6


In [28]:
# reemplazar los valores menores que 45 por np.nan (where es un metodo de pandas)
df2['GLU'] = df2['GLU'].where(df2['GLU'] > 45, np.nan)
nun_missing_3 = (df2['GLU'] < 45).sum()
print(nun_missing_3)

0


In [29]:
df2

,NEMB,GLU,PART,GROS,HUR,BMI,FPRED,AGE,CLASS
0,6,148.0,72,35,0,33.6,0.627,50,1
1,1,85.0,66,29,0,26.6,0.351,31,0
2,8,183.0,64,0,0,23.3,0.672,32,1
3,1,89.0,66,23,94,28.1,0.167,21,0
4,0,137.0,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101.0,76,48,180,32.9,0.171,63,0
764,2,122.0,70,27,0,36.8,0.340,27,0
765,5,121.0,72,23,112,26.2,0.245,30,0
766,1,126.0,60,0,0,30.1,0.349,47,1


## Opcion 1 : Remover filas con valores faltantes

In [30]:
# remover filas con na
df.dropna(inplace=True)
# mirar el resultado final
df.shape

(392, 9)

In [33]:
df

,NEMB,GLU,PART,GROS,HUR,BMI,FPRED,AGE,CLASS
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1
6,3,78.0,50.0,32.0,88.0,31.0,0.248,26,1
8,2,197.0,70.0,45.0,543.0,30.5,0.158,53,1
13,1,189.0,60.0,23.0,846.0,30.1,0.398,59,1
...,...,...,...,...,...,...,...,...,...
753,0,181.0,88.0,44.0,510.0,43.3,0.222,26,1
755,1,128.0,88.0,39.0,110.0,36.5,1.057,37,1
760,2,88.0,58.0,26.0,16.0,28.4,0.766,22,0
763,10,101.0,76.0,48.0,180.0,32.9,0.171,63,0


In [34]:
df.loc[3] #notar que despues de realizar el drop, el index queda roto y .loc[3] se refiere al valor 3

NEMB      1.000
GLU      89.000
PART     66.000
GROS     23.000
HUR      94.000
BMI      28.100
FPRED     0.167
AGE      21.000
CLASS     0.000
Name: 3, dtype: float64

In [35]:
# Reindexar el DataFrame con el índice predeterminado
df_reindexado = df.reset_index(drop=True)

# Mostrar el DataFrame reindexado
df_reindexado

,NEMB,GLU,PART,GROS,HUR,BMI,FPRED,AGE,CLASS
0,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
1,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1
2,3,78.0,50.0,32.0,88.0,31.0,0.248,26,1
3,2,197.0,70.0,45.0,543.0,30.5,0.158,53,1
4,1,189.0,60.0,23.0,846.0,30.1,0.398,59,1
...,...,...,...,...,...,...,...,...,...
387,0,181.0,88.0,44.0,510.0,43.3,0.222,26,1
388,1,128.0,88.0,39.0,110.0,36.5,1.057,37,1
389,2,88.0,58.0,26.0,16.0,28.4,0.766,22,0
390,10,101.0,76.0,48.0,180.0,32.9,0.171,63,0


$$\color{red}{\text{OJO!! Remover las filas solo porque tengan nulos no es la mejor forma de trabajar}}$$

## Opcion 2: Imputar los faltantes

Para esto tenemos varias opciones
1. Introducir un valor constante para los nulos o una categoria llamada Desconocido en variables categoricas
2. Reemplazar por un valor seleccionado al azar de los otros registros
3. Usar la media, mediana o moda para rellenar el valor
4. Valor estimado usando un modelo de ML

In [36]:
url='https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv'
df= pd.read_csv(url,sep=',', header=None)
print(df.shape)
df=df.replace(0, np.nan)
print(df.shape)

(768, 9)
(768, 9)


In [ ]:
print(df.head())

     0      1     2     3      4     5      6   7    8
0  6.0  148.0  72.0  35.0    NaN  33.6  0.627  50  1.0
1  1.0   85.0  66.0  29.0    NaN  26.6  0.351  31  NaN
2  8.0  183.0  64.0   NaN    NaN  23.3  0.672  32  1.0
3  1.0   89.0  66.0  23.0   94.0  28.1  0.167  21  NaN
4  NaN  137.0  40.0  35.0  168.0  43.1  2.288  33  1.0


In [ ]:
# reemplazar con la media
df.fillna(df.mean(), inplace=True)
print(df.shape)
print(df.head().round(2))

(768, 9)
      0      1     2      3       4     5     6   7    8
0  6.00  148.0  72.0  35.00  155.55  33.6  0.63  50  1.0
1  1.00   85.0  66.0  29.00  155.55  26.6  0.35  31  1.0
2  8.00  183.0  64.0  29.15  155.55  23.3  0.67  32  1.0
3  1.00   89.0  66.0  23.00   94.00  28.1  0.17  21  1.0
4  4.49  137.0  40.0  35.00  168.00  43.1  2.29  33  1.0


La librería ```scikit-learn``` proporciona la clase de preprocesamiento ```SimpleImputer``` que se puede usar para reemplazar los valores perdidos.

Es una clase flexible que le permite especificar el valor a reemplazar (puede ser algo diferente a NaN) y la técnica utilizada para reemplazarlo (como media, mediana o moda). La clase ```SimpleImputer``` opera directamente en la matriz NumPy en lugar del DataFrame.

El siguiente ejemplo usa la clase SimpleImputer para reemplazar los valores faltantes con la media de cada columna y luego imprime el número de valores de NaN en la matriz transformada.

In [ ]:
from numpy import nan
from numpy import isnan
from pandas import read_csv
from sklearn.impute import SimpleImputer
url='https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv'
df= pd.read_csv(url,sep=',', header=None)
df=df.replace(0, np.nan)
print(df.shape)
print(df.head())

(768, 9)
     0      1     2     3      4     5      6   7    8
0  6.0  148.0  72.0  35.0    NaN  33.6  0.627  50  1.0
1  1.0   85.0  66.0  29.0    NaN  26.6  0.351  31  NaN
2  8.0  183.0  64.0   NaN    NaN  23.3  0.672  32  1.0
3  1.0   89.0  66.0  23.0   94.0  28.1  0.167  21  NaN
4  NaN  137.0  40.0  35.0  168.0  43.1  2.288  33  1.0


In [ ]:
# Crear un numpy array con los valores
valores = df.values
# definir el imputador
imputador = SimpleImputer(missing_values=np.nan, strategy='mean')
# transformar el dataset
transformados = imputador.fit_transform(valores)
# Contar el numero de NaN en cada columna
print('Missing: ', isnan(transformados).sum())


Missing:  0


In [ ]:
transformados=pd.DataFrame(transformados)
#transformados= transformados.rename(columns={0: "NEMB", 1: "GLU", 2: "PART",3:"GROS",4:"HUR",5:"BMI",6:"FPRED",7:"AGE",8:"CLASS"})
print(transformados.head().round(2))

      0      1     2      3       4     5     6     7    8
0  6.00  148.0  72.0  35.00  155.55  33.6  0.63  50.0  1.0
1  1.00   85.0  66.0  29.00  155.55  26.6  0.35  31.0  1.0
2  8.00  183.0  64.0  29.15  155.55  23.3  0.67  32.0  1.0
3  1.00   89.0  66.0  23.00   94.00  28.1  0.17  21.0  1.0
4  4.49  137.0  40.0  35.00  168.00  43.1  2.29  33.0  1.0


In [ ]:
imputador = SimpleImputer(missing_values=-999, strategy='mean')
# transformar el dataset
transformados = imputador.fit_transform(transformados)
# Contar el numero de NaN en cada columna
print('Missing: ', isnan(transformados).sum())

Missing:  0


In [ ]:
transformados=pd.DataFrame(transformados)
#transformados= transformados.rename(columns={0: "NEMB", 1: "GLU", 2: "PART",3:"GROS",4:"HUR",5:"BMI",6:"FPRED",7:"AGE",8:"CLASS"})
print(transformados.head().round(2))

      0      1     2      3       4     5     6     7    8
0  6.00  148.0  72.0  35.00  155.55  33.6  0.63  50.0  1.0
1  1.00   85.0  66.0  29.00  155.55  26.6  0.35  31.0  1.0
2  8.00  183.0  64.0  29.15  155.55  23.3  0.67  32.0  1.0
3  1.00   89.0  66.0  23.00   94.00  28.1  0.17  21.0  1.0
4  4.49  137.0  40.0  35.00  168.00  43.1  2.29  33.0  1.0


In [ ]:
# Crear un numpy array con los valores
valores = df.values
# definir el imputador
imputador = SimpleImputer(missing_values=np.nan, strategy='median')
# trabsformar el dataset
transformados = imputador.fit_transform(valores)
# Contar el numero de NaN en cada columna
print('Missing: %d' % isnan(transformados).sum())
transformados=pd.DataFrame(transformados)
#transformados= transformados.rename(columns={0: "NEMB", 1: "GLU", 2: "PART",3:"GROS",4:"HUR",5:"BMI",6:"FPRED",7:"AGE",8:"CLASS"})
print(transformados.head().round(2))

Missing: 0
     0      1     2     3      4     5     6     7    8
0  6.0  148.0  72.0  35.0  125.0  33.6  0.63  50.0  1.0
1  1.0   85.0  66.0  29.0  125.0  26.6  0.35  31.0  1.0
2  8.0  183.0  64.0  29.0  125.0  23.3  0.67  32.0  1.0
3  1.0   89.0  66.0  23.0   94.0  28.1  0.17  21.0  1.0
4  4.0  137.0  40.0  35.0  168.0  43.1  2.29  33.0  1.0


In [ ]:
# Crear un numpy array con los valores
valores = df.values
# definir el imputador
imputador = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
# trabsformar el dataset
transformados = imputador.fit_transform(valores)
# Contar el numero de NaN en cada columna
print('Missing: %d' % isnan(transformados).sum())
transformados=pd.DataFrame(transformados)
#transformados= transformados.rename(columns={0: "NEMB", 1: "GLU", 2: "PART",3:"GROS",4:"HUR",5:"BMI",6:"FPRED",7:"AGE",8:"CLASS"})
print(transformados.head().round(2))

Missing: 0
     0      1     2     3      4     5     6     7    8
0  6.0  148.0  72.0  35.0  105.0  33.6  0.63  50.0  1.0
1  1.0   85.0  66.0  29.0  105.0  26.6  0.35  31.0  1.0
2  8.0  183.0  64.0  32.0  105.0  23.3  0.67  32.0  1.0
3  1.0   89.0  66.0  23.0   94.0  28.1  0.17  21.0  1.0
4  1.0  137.0  40.0  35.0  168.0  43.1  2.29  33.0  1.0


In [ ]:
# Crear un numpy array con los valores
valores = df.values
# definir el imputador
imputador = SimpleImputer(missing_values=np.nan, strategy='constant',fill_value=10)
# trabsformar el dataset
transformados = imputador.fit_transform(valores)
# Contar el numero de NaN en cada columna
print('Missing: %d' % isnan(transformados).sum())
transformados=pd.DataFrame(transformados)
#transformados= transformados.rename(columns={0: "NEMB", 1: "GLU", 2: "PART",3:"GROS",4:"HUR",5:"BMI",6:"FPRED",7:"AGE",8:"CLASS"})
print(transformados.head().round(2))

Missing: 0
      0      1     2     3      4     5     6     7     8
0   6.0  148.0  72.0  35.0   10.0  33.6  0.63  50.0   1.0
1   1.0   85.0  66.0  29.0   10.0  26.6  0.35  31.0  10.0
2   8.0  183.0  64.0  10.0   10.0  23.3  0.67  32.0   1.0
3   1.0   89.0  66.0  23.0   94.0  28.1  0.17  21.0  10.0
4  10.0  137.0  40.0  35.0  168.0  43.1  2.29  33.0   1.0


In [ ]:
# Para obtener ayuda sobre la clase
SimpleImputer?

Init signature:
SimpleImputer(
    *,
    missing_values=nan,
    strategy='mean',
    fill_value=None,
    copy=True,
    add_indicator=False,
    keep_empty_features=False,
)
Docstring:     
Univariate imputer for completing missing values with simple strategies.

Replace missing values using a descriptive statistic (e.g. mean, median, or
most frequent) along each column, or using a constant value.

Read more in the :ref:`User Guide <impute>`.

.. versionadded:: 0.20
   `SimpleImputer` replaces the previous `sklearn.preprocessing.Imputer`
   estimator which is now removed.

Parameters
----------
missing_values : int, float, str, np.nan, None or pandas.NA, default=np.nan
    The placeholder for the missing values. All occurrences of
    `missing_values` will be imputed. For pandas' dataframes with
    nullable integer dtypes with missing values, `missing_values`
    can be set to either `np.nan` or `pd.NA`.

strategy : str, default='mean'
    The imputation strategy.

    - If "mean",

El parametro ```strategy``` puede tomar valores de ```['mean','median','most_frequent','constant']``` si se usa ``constant``` se debe proporcionar el argumento ```fill_value````con el valor a reemplazar